In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import os

# Computation of the statical polarizability 

This nb assume that the GS analysis for a set of molecule has been performed. The structure of this working directory consists in a folder for each molecule. Inside each folder there is one logifle (or more if different study varying for instance xc functional or equilibrium position) that contains the result of the GS computation used as input of the present analysis

Scan the path folder and build the molecules_database by looking at the directories in the path

In [40]:
molecules_database = os.listdir('.')
for f in reversed(molecules_database):
    if os.path.isdir(f) == False:
        molecules_database.remove(f)
if os.path.isdir('.ipynb_checkpoints'):
    molecules_database.remove('.ipynb_checkpoints')
if os.path.isdir('alpha'):
    molecules_database.remove('alpha')

molecules_database

['H20', 'CO']

In [41]:
# Choose a molecule to perform the analysis
molecule = molecules_database[1]
molecule

'CO'

In [42]:
gs = lf.Logfile(molecule+'/'+'log-lda.yaml')

Define some useful functions

In [45]:
# may be moved into logfiles?
def get_xc_functional(l):
    """
    Read the field ['dft']['ixc'] of the logfile and return
    the 'LDA' if the result is 1 or 'PBE' if it is equal to 11
    
    Args:
        l (logFile): the logFile
    """
    value = l.log['dft']['ixc']
    if value == 1:
        xc = 'LDA'
    elif value == 11:
            xc = 'PBE'
    else:
        xc = None
    return xc

def append_electric_field_runs(study,intensity,inp):
    """
    Append 6 runs (one for each direction of the field including both positive and negative values) to
    the dataset
    
    Args:
        study (dataset): the dataset
        intensity (float): the field intensity
        inp (inputFile): the input file of the run 
    """
    for ind,sign in enumerate(['+','-']):
        for idir,coord in enumerate(['x','y','z']):
            el=np.zeros(3)
            el[idir]=(1-2*ind)*intensity
            inp.apply_electric_field(el.tolist())
            study.append_run({'rmult':inp['dft']['rmult'][0],'dir':coord,'sign':sign,'F':intensity},code,input=inp)

Define global variables

In [47]:
# Alpha dictionary. At the end of the convergence procedure it will contains the value of the statical
# polarizability associated to the actual value of rmult
alpha = {}
# Field intensity
field_intensity = [1e-2,5e-3,1e-3]

In [48]:
# Define the default parameters of the input file
inp = I.Inputfile()
inp.set_hgrid(gs.log['dft']['hgrids'])
inp.set_xc(get_xc_functional(gs))
inp.set_wavefunction_convergence(gnrm=1.0e-5)
inp

{'dft': {'gnrm_cv': 1e-05, 'hgrids': 0.37, 'ixc': 'LDA'}}

As first step study choose the value of the rmult of the gs and perform an analysis to determine the value of the electric field for computing the statical polarizability

In [49]:
inp.set_rmult(gs.log['dft']['rmult'])
inp

{'dft': {'gnrm_cv': 1e-05, 'hgrids': 0.37, 'ixc': 'LDA', 'rmult': [7.0, 9.0]}}

Define the dataset for the evaluation of the statical polarizability

In [53]:
posinp_file = molecule+'/'+molecule+'_posinp.xyz'
print posinp_file

code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True)
study = D.Dataset(label='alpha',run_dir='alpha',posinp=posinp_file)

CO/CO_posinp.xyz
Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


In [54]:
for f in field_intensity[:2]:
    append_electric_field_runs(study,f,inp)

In [55]:
ind = 9
write(study.ids[ind])
write(study.runs[ind])

{'rmult': 7.0, 'F': 0.005, 'dir': 'x', 'sign': '-'}
{'run_dir': 'alpha', 'input': {'dft': {'ixc': 'LDA', 'gnrm_cv': 1e-05, 'elecfield': [-0.005, 0.0, 0.0], 'hgrids': 0.37, 'rmult': [7.0, 9.0]}}, 'posinp': 'CO/CO_posinp.xyz', 'label': 'alpha'}


Can we define eval_alpha_one_field as a post processing functions? In this case it would be better to append run associated to a single field value, extract and store alphas[F] and then remove the appended run and start again with the new value of the field...Otherwise a new instance of study can be created for each value of the field intensity

In [56]:
# how to avoid the high-verbose mode?
study.run()

Copy the posinp file 'CO/CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:x,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO/CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:y,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:y,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO/CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:z,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:z,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO/CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:-.yaml"
Run directory alpha


{0: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa4ef368>,
 1: <BigDFT.Logfiles.Logfile instance at 0x7f6aaafb3098>,
 2: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa77c320>,
 3: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa6ed0e0>,
 4: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa74ed40>,
 5: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa502638>,
 6: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa502998>,
 7: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa3495f0>,
 8: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa30a170>,
 9: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa30a0e0>,
 10: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa30a200>,
 11: <BigDFT.Logfiles.Logfile instance at 0x7f6aaa30a1b8>}

In [57]:
def eval_alpha_one_field(dipoles,f):
    """"
    Compute alpha given one value of the field intensity and the associated list of 
    six dipoles
    """
    alpha=np.mat(np.zeros(9)).reshape(3,3)
    for ind in range(3):
        alpha[ind] = np.array(dipoles[ind])-np.array(dipoles[ind+3])
    alpha = alpha.T / (2.0*f)
    return alpha

Introduce the alphas dictionary that contains the polarizability tensors for each value of the field intensity. Needed for the convergence study

In [58]:
alphas = {}

In [59]:
for f in field_intensity[:2]:
    dipoles = study.fetch_results({'F':f},attribute = 'dipole')
    alphas[f] = eval_alpha_one_field(dipoles,f)
alphas

{0.005: matrix([[ 1.3211721e+01, -4.5186718e-04, -2.4358770e-05],
         [-4.5186719e-04,  1.3211721e+01, -2.4358770e-05],
         [ 1.0600000e-04,  1.0600000e-04,  1.8530906e+01]]),
 0.01: matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
         [-1.279955e-05,  1.323614e+01, -3.389697e-05],
         [-1.125000e-04, -1.125000e-04,  1.856809e+01]])}

Perform a comparison among the alpha computed with different field and perform a further computation with a field
equal to 1e-3 if the alphas differ more than a given tolerance. The comparison is performed with the allcloe functions that returns true if

abs(a-b) < atol + rtol*abs(b)

so we chose atol = 1e-3 and rtol = 1e-2. So for small elements that are deeply affected by numerical noise the
maximum allowed discrepacy is atol while for the relevan ones the percentage difference is rtol

In [61]:
at = 1e-3
rt = 1e-2

if the comparison returns False we need to perform a further computation for the lowest value of the field intensity

In [62]:
convergence = np.allclose(alphas[field_intensity[0]],alphas[field_intensity[1]],atol = at, rtol = rt)
if convergence:
    write('Convergence achieved for field intensity ',field_intensity[0])
    alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[0]]
else:
    write('Convergence for field intensity ',field_intensity[0],' failed')
    write('Reduce the value of the field')
    append_electric_field_runs(study,field_intensity[2],inp)
    study.run()
    dipoles = study.fetch_results({'F':field_intensity[2]},attribute = 'dipole')
    alphas[field_intensity[2]] = eval_alpha_one_field(dipoles,field_intensity[2])
    convergence = np.allclose(alphas[field_intensity[0]],alphas[field_intensity[1]],atol = at, rtol = rt)
    if convergence:
        write('Convergence achieved for field intensity ',field_intensity[1])
        alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[1]]
    else:
        write('Convergence for field intensity ',field_intensity[1],' failed')
        write('Return the value of alpha associated to ',field_intensity[1],'. Perform further check!!!')
        alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[1]]

Convergence achieved for field intensity  0.01


In [63]:
alpha

{7.0: matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
         [-1.279955e-05,  1.323614e+01, -3.389697e-05],
         [-1.125000e-04, -1.125000e-04,  1.856809e+01]])}

In [ ]:
#############################################################

The aim of this nb is to provide all the tools needed to compute the statical polarizability of a molecule.
The results of this analysis allow us to perform convergence study of the statical polarizability w.r.t the size of the simulation domain and to check if the strenght of the field used to perturb the ground state configuration of the molecule is compatible with the linear response regime 

The data structure which encodes all the computation that will be performed during the analysis is a python dictionary generically called molecule with structure:
* molecule[rmult][field]

where [field] is a tuple with the cartesian components of the statical field

In order to perform this analysis several methods need to be defined, in particular:
 * computeStatPol(molecule,fieldNorm,box) : take as inputs the properties of the choses molecule, the value of the field norm and the value of the box size (rmult in our specific case). This method check if the results associated to the given field norm and box value are present in molecule. If needed perform the computation storing the results in molecule and add the 'statPol' key to molecule with the (matrix valued) result for the statical polarizability

Let us start this analysis with some specific examples